In [11]:
import os
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping
import matplotlib.pyplot as plt

# =========================
# Wczytanie i przygotowanie danych
# =========================
df = pd.read_csv(
    "european_capitals_history_clean.csv",
    parse_dates=["date"]
)

df_city = df[df['city'] == 'Warszawa']
df_city = df_city[['date', 'city', 'tavg']]
df_city.sort_values(by=["date"], inplace=True)

# =========================
# Tworzenie sekwencji czasowych
# =========================
def create_sequences(df, window=5):
    sequences = []
    labels = []
    for i in range(len(df) - window):
        seq = df.iloc[i:i + window]['tavg'].values
        label = df.iloc[i + window]['tavg']
        sequences.append(seq)
        labels.append(label)
    return np.array(sequences), np.array(labels)

sequences, labels = create_sequences(df_city, window=5)

# =========================
# Normalizacja danych
# =========================
scaler = MinMaxScaler()
sequences_scaled = scaler.fit_transform(sequences.reshape(-1, 1)).reshape(sequences.shape)
labels_scaled = scaler.transform(labels.reshape(-1, 1))
sequences_scaled = sequences_scaled.reshape((sequences_scaled.shape[0], sequences_scaled.shape[1], 1))

# =========================
# Podział na zbiór treningowy i testowy
# =========================
train_size = int(len(sequences_scaled) * 0.8)
X_train = sequences_scaled[:train_size]
X_test  = sequences_scaled[train_size:]
y_train = labels_scaled[:train_size]
y_test  = labels_scaled[train_size:]

# =========================
# Budowa modelu LSTM z dwiema warstwami
# =========================
model = Sequential()
model.add(LSTM(units=64, return_sequences=True, input_shape=(X_train.shape[1], 1)))
model.add(Dropout(0.2))
model.add(LSTM(units=32, return_sequences=False))
model.add(Dense(1))

model.compile(optimizer='adam', loss='mean_squared_error')

# =========================
# EarlyStopping
# =========================
early_stop = EarlyStopping(
    monitor='val_loss',
    patience=10,
    restore_best_weights=True
)

# =========================
# Trenowanie modelu
# =========================
history = model.fit(
    X_train,
    y_train,
    epochs=100,           # można zwiększyć liczbę epok, EarlyStopping zatrzyma trening wcześniej
    batch_size=32,
    validation_data=(X_test, y_test),
    callbacks=[early_stop]
)

# =========================
# Zapis modelu
# =========================
model_dir = "models"
os.makedirs(model_dir, exist_ok=True)
model_path = os.path.join(model_dir, "lstm_warszawa_2layers.keras")
model.save(model_path)
print(f"Model zapisany w: {model_path}")

# =========================
# Wizualizacja straty
# =========================
plt.plot(history.history['loss'], label='Trening')
plt.plot(history.history['val_loss'], label='Walidacja')
plt.legend()
plt.title("Strata modelu podczas treningu")
plt.xlabel("Epoka")
plt.ylabel("Strata")
plt.show()

# =========================
# Predykcja na zbiorze testowym
# =========================
predictions = model.predict(X_test)
predictions_rescaled = scaler.inverse_transform(predictions)
y_test_rescaled = scaler.inverse_transform(y_test)

plt.plot(df_city['date'].iloc[-len(y_test_rescaled):], y_test_rescaled, label='Rzeczywiste wartości')
plt.plot(df_city['date'].iloc[-len(predictions_rescaled):], predictions_rescaled, label='Prognozy')
plt.legend()
plt.title("Porównanie prognoz i rzeczywistych wartości — LSTM 2 warstwy")
plt.xlabel("Data")
plt.ylabel("Średnia temperatura")
plt.xticks(rotation=45)
plt.grid(True)
plt.tight_layout()
plt.show()


In [12]:
from sklearn.model_selection import TimeSeriesSplit
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping
import numpy as np

# =========================
# Funkcja budująca model LSTM z 2 warstwami i Dropout
# =========================
def build_lstm_model(input_shape):
    model = Sequential()
    # Pierwsza warstwa LSTM zwracająca sekwencję
    model.add(LSTM(units=64, return_sequences=True, input_shape=input_shape))
    model.add(Dropout(0.2))
    # Druga warstwa LSTM zwracająca tylko ostatnią wartość
    model.add(LSTM(units=32, return_sequences=False))
    # Warstwa wyjściowa
    model.add(Dense(1))
    # Kompilacja
    model.compile(optimizer='adam', loss='mean_squared_error')
    return model

# =========================
# Time Series Cross-Validation
# =========================
tscv = TimeSeriesSplit(n_splits=5)

rmse_scores = []
mae_scores  = []
mse_scores  = []
r2_scores   = []

fold = 1

for train_index, test_index in tscv.split(sequences_scaled):
    print(f"\nFold {fold}")

    X_train = sequences_scaled[train_index]
    X_test  = sequences_scaled[test_index]

    y_train = labels_scaled[train_index]
    y_test  = labels_scaled[test_index]

    model = build_lstm_model(input_shape=(X_train.shape[1], 1))

    # =========================
    # EarlyStopping
    # =========================
    early_stop = EarlyStopping(
        monitor='val_loss',
        patience=10,
        restore_best_weights=True
    )

    model.fit(
        X_train,
        y_train,
        epochs=100,           # EarlyStopping zatrzyma trening wcześniej
        batch_size=32,
        verbose=0,
        validation_data=(X_test, y_test),
        callbacks=[early_stop],
        shuffle=False
    )

    y_pred = model.predict(X_test)

    # Odwrócenie skalowania
    y_test_rescaled = scaler.inverse_transform(y_test)
    y_pred_rescaled = scaler.inverse_transform(y_pred)

    mse  = mean_squared_error(y_test_rescaled, y_pred_rescaled)
    rmse = np.sqrt(mse)
    mae  = mean_absolute_error(y_test_rescaled, y_pred_rescaled)
    r2   = r2_score(y_test_rescaled, y_pred_rescaled)

    mse_scores.append(mse)
    rmse_scores.append(rmse)
    mae_scores.append(mae)
    r2_scores.append(r2)

    print(
        f"R²: {r2:.4f} | "
        f"MAE: {mae:.4f} | "
        f"MSE: {mse:.4f} | "
        f"RMSE: {rmse:.4f}"
    )

    fold += 1

# =========================
# Podsumowanie wyników
# =========================
print("\nWalk-Forward Cross-Validation — LSTM (2 warstwy + EarlyStopping)")
print(f"R²   : {np.mean(r2_scores):.4f}")
print(f"MAE  : {np.mean(mae_scores):.4f}")
print(f"MSE  : {np.mean(mse_scores):.4f}")
print(f"RMSE : {np.mean(rmse_scores):.4f}")


In [13]:
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

# =========================
# Predykcja na zbiorze testowym
# =========================
predictions = model.predict(X_test)

# Odwrócenie skalowania
predictions_rescaled = scaler.inverse_transform(predictions)
y_test_rescaled = scaler.inverse_transform(y_test)

# Dopasowanie długości (czasem w LSTM występuje różnica długości)
min_len = min(len(y_test_rescaled), len(predictions_rescaled))
y_test_rescaled = y_test_rescaled[:min_len]
predictions_rescaled = predictions_rescaled[:min_len]

# =========================
# Obliczenie residuals
# =========================
residuals = y_test_rescaled - predictions_rescaled

# =========================
# Wykres residuals w czasie
# =========================
plt.figure(figsize=(12,6))
plt.plot(
    df_city['date'].iloc[-len(residuals):],
    residuals,
    marker='o',
    linestyle='-',
    color='b'
)
plt.axhline(y=0, color='r', linestyle='--')
plt.title('Residuals modelu LSTM (2 warstwy) — błąd predykcji w czasie')
plt.xlabel('Data')
plt.ylabel('Błąd predykcji (°C)')
plt.grid(True)
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

# =========================
# Histogram residuals
# =========================
plt.figure(figsize=(10,6))
sns.histplot(residuals, kde=True, color='b')
plt.title('Rozkład residuals modelu LSTM (2 warstwy)')
plt.xlabel('Błąd predykcji (°C)')
plt.ylabel('Częstość')
plt.grid(True)
plt.tight_layout()
plt.show()
